In [34]:
import pickle
import gc
import datetime
from math import floor, ceil
from time import perf_counter

import numpy as np  
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns;   sns.set_style("whitegrid")

import importlib
import CVA_Module ; importlib.reload(CVA_Module)

from numba import njit
from tqdm.notebook import trange, tqdm_notebook ; tqdm_notebook.pandas()

In [35]:
# Set general font size and Chart Dimentions
plt.rcParams["figure.autolayout"] = True 
plt.rcParams['font.size']       = '18'
CHART_DIMENTIONS_SINGLE_CHART   = (20,7) 
CHART_DIMENTIONS_DOUBLE_CHART   = (20,11) 
SAVE_CHARTS                     = False 

In [36]:
CVA = CVA_Module.CVA(CHART_DIMENTIONS_SINGLE_CHART=CHART_DIMENTIONS_SINGLE_CHART) # initialize CVA Object to be used

## LOAD RATES DATA

In [37]:
swaps_set = pd.read_excel('data/swap_valuation/contracts.xlsx', index_col=0)
scenarios = pd.read_excel('data/rates/scenarios_description.xlsx', index_col=0)

In [38]:
df_SOFR3m_last = pd.read_excel('data/rates/Scenario 1/interpolated_forward_rates_5_Scenario 1.xlsx', index_col=0)

## LOAD SWAP CHARACTERISTICS

In [39]:
def get_swap_value(CVA, Notional:int, Floating_rate: list, 
                   RFR_discounted_for_float_list:list, RFR_discounted_for_fixed_list: list,
                  Fixed_rate:float, T:int, V_for_Float_Leg, df_SOFR3m_last, freq=0.25, dt=1/52):

    """ Step (5.2) : Re evaluate the swap for the given paths """
    IRS_value_list = []
    dt      = 1/52
    t_array = np.arange(start=0, stop=T+dt, step=dt)

    for i in tqdm_notebook(range(len(rfr_float_P_t_T_list))):   
        npv_floating, npv_fixed, sum_npv_floating, sum_npv_fix, IRS_value = CVA.interest_rate_swap_valuation_vectorised(
            Notional = Notional,
            Floating_rate = Floating_rate[i],            # <- vector
            Floating_rate_cashflows_frequency = freq, 
            RFR_discounted_for_float_list = RFR_discounted_for_float_list[i],  # <- vector
            RFR_discounted_for_fixed_list = RFR_discounted_for_fixed_list[i],  # <- vector
            Fixed_rate = Fixed_rate,
            Fixed_rate_cashflows_frequency = freq,
            V_for_Float_Leg=V_for_Float_Leg
            )    
        IRS_value_list.append(IRS_value * np.exp(-df_SOFR3m_last.iloc[i, 0] * i*dt)) # Продисконтировано до даты оценки
        #IRS_value_list.append(IRS_value) # На форвардную дату
    
    print(len(IRS_value_list))
    
    IRS_values_df = pd.DataFrame(IRS_value_list, index=t_array)
    
    return IRS_values_df

In [40]:
swap_values_dict_all = {}
for scen in scenarios.columns:
    swap_values_dict = {}
    for swap_n in swaps_set.columns:
        # Discount Factor For fixed rate leg - rfr_fixed_P_t_T_list
        with open(f'data/rates/{scen}/DF_fixed_leg_{int(swaps_set.loc["T", swap_n])}_{scen}.pickle', 'rb') as f:
            rfr_fixed_P_t_T_list = pickle.load(f)

        # Discount Factors for floating rate leg - rfr_float_P_t_T_list
        with open(f'data/rates/{scen}/DF_float_leg_{int(swaps_set.loc["T", swap_n])}_{scen}.pickle', 'rb') as f:
            rfr_float_P_t_T_list = pickle.load(f)

        # Forward Rate Curve for all future dates - forward_rate_curves_list
        with open(f'data/rates/{scen}/ForwardRateCurvesProjection_{int(swaps_set.loc["T", swap_n])}_{scen}.pickle', 'rb') as f:
            forward_rate_curves_list = pickle.load(f)         
            
        # ATM Swap Rate
        with open(f'data/rates/{scen}/swap_rate_inputs_{int(swaps_set.loc["T", swap_n])}_{scen}.pickle', 'rb') as f:
            atm_swap_rate = pickle.load(f)    

        print(f"Swap rate of {swap_n} = {atm_swap_rate['swap_rate_atm']}")
        # Value Swap
        swap_values = get_swap_value(CVA, Notional=swaps_set.loc['notional', swap_n],
                                     Floating_rate=forward_rate_curves_list,
                                     RFR_discounted_for_float_list=rfr_float_P_t_T_list,
                                     RFR_discounted_for_fixed_list=rfr_fixed_P_t_T_list,
                                     Fixed_rate=swaps_set.loc['Fixed_rate', swap_n],
                                     T=swaps_set.loc['T', swap_n],
                                     V_for_Float_Leg=swaps_set.loc['v_for_fixed_leg', swap_n],
                                     df_SOFR3m_last=df_SOFR3m_last)


        swap_values_dict[swap_n] = swap_values
    
    swap_values_dict_all[scen] = swap_values_dict

Swap rate of SWAP_1 = 0.04259437856198282


  0%|          | 0/261 [00:00<?, ?it/s]

261
Swap rate of SWAP_2 = 0.04259437856198282


  0%|          | 0/261 [00:00<?, ?it/s]

261
Swap rate of SWAP_3 = 0.04479496695091921


...

261
Swap rate of SWAP_2 = 0.04259437856198282


  0%|          | 0/261 [00:00<?, ?it/s]

261
Swap rate of SWAP_3 = 0.04479496695091921


  0%|          | 0/157 [00:00<?, ?it/s]

157


In [41]:
def max_size_of_frame(swap_values_dict, return_index=True):
    maximum = 0
    for i in swap_values_dict.keys():
        if len(swap_values_dict[i]) > maximum:
            maximum = len(swap_values_dict[i])
            swap_name = i
    if return_index:
        return swap_values_dict[swap_name].index
    else:
        return swap_name

In [42]:
for scen in scenarios.columns:
    max_sized_frame = pd.DataFrame(index=max_size_of_frame(swap_values_dict_all[scen]))    
    for swap_n in swaps_set.columns:
        
        swap_values_dict_all[scen][swap_n] = max_sized_frame.merge(swap_values_dict_all[scen][swap_n],
                                                                   how='left',
                                                                   left_index=True,
                                                                   right_index=True).fillna(0)

### Calculate Netting Sets

In [43]:
for scen in scenarios.columns:
    # Netting Set 1 + 2
    swap_values_dict_all[scen]['NS_1'] = swap_values_dict_all[scen]['SWAP_1'] #+ swap_values_dict_all[scen]['SWAP_2']
    
    # Netting Set 1 + 3
    swap_values_dict_all[scen]['NS_2'] = swap_values_dict_all[scen]['SWAP_1'] + swap_values_dict_all[scen]['SWAP_2']
    
    # Netting Set 1 + 2 + 3
    swap_values_dict_all[scen]['NS_3'] = (swap_values_dict_all[scen]['SWAP_1'] + swap_values_dict_all[scen]['SWAP_3'])
#                                           + 
#                                           swap_values_dict_all[scen]['SWAP_2'] + 
#                                           swap_values_dict_all[scen]['SWAP_3'])

In [44]:
# Calculated SWAP Values for given Contracts Conditions
with open(f'data/swap_valuation/swap_values_dict.pickle', 'wb') as f:
    pickle.dump(swap_values_dict_all, f)